# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Степанов Николай

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [0]:
import numpy as np
import pandas as pd
import collections

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [205]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-26 09:26:25--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-26 09:26:26--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf32d22825ee9c42cc0fd376947.dl.dropboxusercontent.com/cd/0/inline/A2nFXq8Zrv40B6nZ0nPNAYsX05ofmO2m45UaGkneLpwFBOxBh3U8wO7WtlRCHQ6eekpua14iaj9827Gbooq8_uHWZ7ahvqOOVeJBmAYKaol2fGLrwBpYTpm98qFUEjRFKkk/file# [following]
--2020-04-26 09:26:26--  https://ucf32d22825ee9c42cc0fd376947.dl.dropboxusercontent.com/cd/0/inline/A2nFXq8Zrv40B6nZ0nPNAYsX05ofmO2m45UaGkneLpwFBOxBh3U8wO7WtlRCHQ6eekpua14iaj9827Gbooq8_uHWZ7ahvqOOVeJ

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [207]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [209]:
data.shape

(30000, 4)

In [0]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [208]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [211]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [0]:
vectPreprocess = np.vectorize(preprocess)
X_train = vectPreprocess(X_train)
X_test = vectPreprocess(X_test)

In [213]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [0]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

In [0]:
bow_vocabulary = collections.Counter()

for i in range(len(X_train[:])):
     for word in (X_train[i][0].split() + X_train[i][1].split()):
         bow_vocabulary[word] += 1   

In [0]:
k = 10000
bow_vocabulary = dict(bow_vocabulary.most_common(k))

In [0]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [0]:
words = np.array(list(bow_vocabulary.keys()))

In [0]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    bow = np.zeros(k)
    for i in range(k):
        bow[i] = text.split().count(words[i])
    return np.array(bow)

In [0]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [0]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    bow = np.zeros((len(items), k))
    for i, descr in zip(range(len(items)), items):
      bow[i] = text_to_bow(descr[1])
    
    return np.array(bow)

In [0]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [0]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [226]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7011111111111111


In [227]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6843333333333333


### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
  def items_to_bow_title(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    bow = np.zeros((len(items), k))
    for i, descr in zip(range(len(items)), items):
      bow[i] = text_to_bow(descr[0])
    
    return np.array(bow)

In [0]:
coeff = 1.5
X_train_bow = X_train_bow + coeff*items_to_bow_title(X_train)
X_test_bow = X_test_bow + coeff*items_to_bow_title(X_test)

In [230]:
bow_model_LR = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model_LR.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7915555555555556


In [231]:
bow_model_SVC = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model_SVC.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7605555555555555


**Качество работы алгоритма улучшилось, так как мы добавили новую информацию о товаре, т.е. сделали признаки, на которых обучается модель, еще более индивидуальным .**

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [0]:
from sklearn import preprocessing

In [0]:
normalized_X_train_bow = preprocessing.normalize(X_train_bow)
normalized_X_test_bow = preprocessing.normalize(X_test_bow)

In [234]:
bow_model_LR = LogisticRegression(max_iter=100).fit(normalized_X_train_bow, y_train)
print(accuracy_score(bow_model_LR.predict(normalized_X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.6942222222222222


In [235]:
bow_model_SVC = LinearSVC(max_iter=70).fit(normalized_X_train_bow, y_train)
print(accuracy_score(bow_model_SVC.predict(normalized_X_test_bow), y_test))

0.8053333333333333


**Качество алгоритма SVC улучшилось, так как мы загнали наши признаки в диапазон [0, 1] - именно к этому он и чувствителен (нормализация). Т.е. теперь важность каждого признака будет учтена. Работа же LogReg ухуджиласть, так как данный алгоритм требует стандартизации данных, т.е. чтобы среднее было равно 0. Также, возможно, что алгоритму не хватает кол-во итераций, чтобы хорошо обучиться на нормализованных данных (теперь все данные для него равносильны и вносят один вклад)**

In [236]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_bow)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [237]:
bow_model_LR = LogisticRegression(max_iter=100).fit(scaler.transform(X_train_bow), y_train)
print(accuracy_score(bow_model_LR.predict(scaler.transform(X_test_bow)), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7352222222222222


**Видим, что стандартизация для LogReg работаем лучше, чем нормализация, но попрежнему не хватает кол-во итераций для обучения**

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [238]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-26 10:05:34--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-26 10:05:35--  http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)... 5.45.220.14, 2a02:6b8:0:2002::15
Connecting to cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)|5.45.220.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [239]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-bl250tq2
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-bl250tq2
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=d4beb99198fcd256e60d6ad3db036c47ee54d33c6b8a7165e3ba481b6fafe72c
  Stored in directory: /tmp/pip-ephem-wheel-cache-msdhuezr/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [0]:
from pymystem3 import Mystem

In [0]:
mystem_tok = Mystem()

def preprocessMystem(text: str) -> str:
    return ''.join(mystem_tok.lemmatize(text.lower()))


In [0]:
vectPreprocessMystem = np.vectorize(preprocessMystem)
X_train = vectPreprocessMystem(X_train)
X_test = vectPreprocessMystem(X_test)

In [243]:
X_train[:5]

array([['сапог 46 размер новый\n', 'сапог 46 размер новый\n'],
       ['светильник потолочный swarovski\n',
        'светильник потолочный swarovski 6 штука , цена за штука . в эксплуатация 2 год , продаваться в связь со смена интерьер в квартира\n'],
       ['iphone 7 plus 128gb red красный в наличие\n',
        '/ / данный цена только для подписчик instagram : iqmac / / новый красный айфон 7 plus в наличие это элегантный и мощный смартфон , который готовый в полный мера раскрывать новый возможность ios 10 . аппарат с 4 - ядерный процессор а10 и 3 гб озу с легкость решать самый ресурсоемкий задача , позволять наслаждаться быстродействие « тяжелый » приложение и игра на 5 , 5 - дюймовый дисплей . аппарат получать экран , как у ipad pro , так что картинка теперь соответствовать кинематографический стандарт .\n'],
       ['пион ирис ромашка рассада\n',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / расти у мы более 70 год / розовый , бордовый и белый / на фото цветок 2

In [0]:
bow_vocabulary = collections.Counter()

for i in range(len(X_train[:])):
    for word in (X_train[i][0].split() + X_train[i][1].split()):
        bow_vocabulary[word] += 1   

In [0]:
bow_vocabulary = dict(bow_vocabulary.most_common(k))
words = np.array(list(bow_vocabulary.keys()))

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [0]:
coeff = 1.5
X_train_bow = X_train_bow +  coeff*items_to_bow_title(X_train)
X_test_bow = X_test_bow + coeff*items_to_bow_title(X_test)

In [0]:
normalized_X_train_bow_mystem = preprocessing.normalize(X_train_bow)
normalized_X_test_bow_mystem = preprocessing.normalize(X_test_bow)

In [249]:
bow_model_LR = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model_LR.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8121111111111111


In [250]:
bow_model_SVC = LinearSVC(max_iter=70).fit(normalized_X_train_bow_mystem, y_train)
print(accuracy_score(bow_model_SVC.predict(normalized_X_test_bow_mystem), y_test))

0.8215555555555556


**Качество модели улучшилось с помощью mystem. Возможно, это связано с тем, что слова привели к единой форме (не ращличаются за счет окончаний, суффиксов и т.д.).**

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [0]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

count_arr = np.zeros(k)

for i in range(len(X_train[:])):
   set_words = set(X_train[i][0].split()+X_train[i][1].split())
   for j in range(k):
    if words[j] in set_words:
      count_arr[j] += 1

In [0]:
counter = collections.Counter()

for i in range(len(X_train[:])):
     for word in (X_train[i][0].split() + X_train[i][1].split()):
         counter[word] += 1
all_words = list(counter.keys())

In [0]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    idf = np.zeros(k)
    tf = np.zeros(k)

    len_doc = len(text.split())
    len_docs = X_train.shape[0]
    
    for i in range(k):
      tf[i] = text.split().count(words[i])/len_doc
      idf[i] = np.log(len_docs/count_arr[i])

    return idf*tf


In [0]:
def items_to_tfidf(items: np.array) -> np.array:
    tfidf = np.zeros((len(items), k))
    for i, descr in zip(range(len(items)), items):
      tfidf[i] = text_to_tfidf(descr[1])
    
    return np.array(tfidf)

In [0]:
  def items_to_tfidf_title(items: np.array) -> np.array:
    tfidf = np.zeros((len(items), k))
    for i, descr in zip(range(len(items)), items):
      tfidf[i] = text_to_tfidf(descr[0])
    
    return np.array(tfidf)

In [0]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

In [0]:
coeff = 1.5
X_train_tfidf = X_train_tfidf + coeff*items_to_tfidf_title(X_train)
X_test_tfidf = X_test_tfidf + coeff*items_to_tfidf_title(X_test)

In [0]:
# Нормализуйте данные
normalized_X_train_tfidf = preprocessing.normalize(X_train_tfidf)
normalized_X_test_tfidf = preprocessing.normalize(X_test_tfidf)

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [259]:
bow_model_LR = LogisticRegression(max_iter=100).fit(normalized_X_train_tfidf, y_train)
print(accuracy_score(bow_model_LR.predict(normalized_X_test_tfidf), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7697777777777778


In [260]:
bow_model_SVC = LinearSVC(max_iter=70).fit(normalized_X_train_tfidf, y_train)
print(accuracy_score(bow_model_SVC.predict(normalized_X_test_tfidf), y_test))

0.8095555555555556


### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer

In [0]:
X_train_new = [X_train[i][0]+ ' ' + X_train[i][1] for i in range(X_train.shape[0])]
X_test_new = [X_test[i][0]+ ' ' + X_test[i][1] for i in range(X_test.shape[0])]

In [0]:
vectorizer = HashingVectorizer(ngram_range=(1, 3), n_features=50000)
X_trainHV = vectorizer.fit_transform(X_train_new)
X_testHV = vectorizer.fit_transform(X_test_new)

In [264]:
bow_model_LR = LogisticRegression(max_iter=100).fit(X_trainHV, y_train)
print(accuracy_score(bow_model_LR.predict(X_testHV), y_test))

0.706


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [265]:
bow_model_SVC = LinearSVC(max_iter=70).fit(X_trainHV, y_train)
print(accuracy_score(bow_model_SVC.predict(X_testHV), y_test))

0.8063333333333333


### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [266]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-26 11:39:22--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-26 11:39:22--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbbb7e829b7fa92064435a22a95.dl.dropboxusercontent.com/cd/0/inline/A2mV0ayYGx_Dmn9mVpdpX2NfwoA58EcpNi_HEXuXCwBydodUQTB36X9dDZgbsu_-trWILR51cBaUhyUzGWB3InVOJPV2vMNjg08H7BRrPHloVw/file# [following]
--2020-04-26 11:39:23--  https://ucbbb7e829b7fa92064435a22a95.dl.dropboxusercontent.com/cd/0/inline/A2mV0ayYGx_Dmn9mVpdpX2NfwoA58EcpNi_HEXuXCwBydodUQTB36X9dDZgbsu_-trWILR51cBaUhyUzGWB3InVOJPV2vMNjg08H7BRrPHloVw/file
Resolving ucbbb7e829b7fa9

In [267]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [0]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['продам'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [0]:
def items_to_emb(items: np.array) -> np.array:
    embedding = np.zeros((len(items), 300))
    for i, descr in zip(range(len(items)), items):
      embedding[i] = sentence_embedding(descr[1])
    
    return np.array(embedding)

In [0]:
X_train_embedding = items_to_emb(X_train)
X_test_embedding = items_to_emb(X_test)

In [273]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)
bow_model_LR = LogisticRegression(max_iter=100).fit(X_train_embedding, y_train)
print(accuracy_score(bow_model_LR.predict(X_test_embedding), y_test))

bow_model_SVC = LinearSVC(max_iter=70).fit(X_train_embedding, y_train)
print(accuracy_score(bow_model_SVC.predict(X_test_embedding), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5338888888888889
0.526


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

![title](d.png)